# AQI data collection
This notebook contains the code to extarct data from the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. The data provided by this API is historic and  not real-time air quality data. The [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

A large part of this code was adapted from the code provided by Dr.David McDonald and is licensed under the the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).






## Importing required dependencies

If you happen to be missing any of these, they can be downloaded using the ```pip <library-name>``` command

In [ ]:
# These are standard python modules
import json, copy, time

# The 'requests','tqdm', and 'pandas' modules are not standard python libraries
# You will need to install these with pip/pip3 if you do not already have it

# distribution module for making web requests
import requests
# a highly recommended progress bar for loops.
from tqdm.auto import tqdm, trange
# a recommended module to read and wrangle data
import pandas as pd
# recommended modules for visualization
import matplotlib.pyplot as plt
import seaborn as sns


# Best for array manipulation
import numpy as np


## Defining constants

All the constants to be used in the rest of the code are defined here

In [ ]:

#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are some of the 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
#
AQS_REQUEST_TEMPLATE = {
    "email":      "chakim28@uw.edu",
    "key":        "",
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}



In [ ]:
# Defining credentials for API calls
USERNAME = "chakim28@uw.edu"
APIKEY = ""  # Place your API key here.


## Making a sign-up request

This code only needs to be exceuted once in order to receive a confirmation email along with a key. Make sure to comment out the code block once the sign-up request goes through, unless you lost your key and need a new one.


In [ ]:
#
#    This implements the sign-up request. The parameters are standardized so that this function definition matches
#    all of the others. However, the easiest way to call this is to simply call this function with your preferred
#    email address.
#
def request_signup(email_address = None,
                   endpoint_url = API_REQUEST_URL,
                   endpoint_action = API_ACTION_SIGNUP,
                   request_template = AQS_REQUEST_TEMPLATE,
                   headers = None):

    # Make sure we have a string - if you don't have access to this email addres, things might go badly for you
    if email_address:
        request_template['email'] = email_address

    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_signup()'")

    if '@' not in request_template['email']:
        raise Exception(f"Must supply an email address to call 'request_signup()'. The string '{request_template['email']}' does not look like an email address.")

    # Compose the signup url - create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response




In [ ]:

  #  A SIGNUP request is only to be done once, to request a key. A key is sent to that email address and needs to be confirmed with a click through
  #  This code should probably be commented out after you've made your key request to make sure you don't accidentally make a new sign-up request

# print("Requesting SIGNUP ...")
# USERNAME = "chakim28@uw.edu"
# response = request_signup(USERNAME)
# print(json.dumps(response,indent=4))
# #

The response to the signup request has the following format:

    {
        "Header": [
            {
                "status": "Success",
                "request_time": "2023-08-07T17:03:27-04:00",
                "url": "https://aqs.epa.gov/data/api/signup?email=chakim28@uw.edu"
            }
        ],
        "Data": [
            "You should receive a registration confirmation email with a link for confirming your email shortly."
        ]
    }



## Example 2. Making a list request
After obtaining your API key, you can access detailed information about air quality monitoring stations and their sensor configurations. The EPA's monitoring infrastructure is dynamic, with frequent updates to both station locations and sensor deployments

The API provides real-time access to monitoring system changes through endpoint requests. This dynamic approach ensures users always have access to current information, rather than relying on static documentation.

We get a list of parameters and their definitions which we could use to decide what data we would like to pull

In [ ]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL,
                      endpoint_action = API_ACTION_LIST_CLASSES,
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):

    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key

    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [ ]:
#
#   The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clustors of
#   sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the
#   a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from
#   that specific sensor.
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

From the list, the most relevant parameter seems to be "AQI Pollutants". We can make another list request to find what details "AQI pollutants" contains

In [ ]:
#
#   Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group)
#   The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
#   We'll use that to make another list request.
#
AQI_PARAM_CLASS = "AQI POLLUTANTS"


In [ ]:
#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


The API enforces a limit of 5 sensor parameters per data request. This restriction affects queries for Air Quality Index (AQI) data, as it's not possible to retrieve all AQI parameters in a single API call.
<br>
To accommodate this limitation, sensor parameters are divided into two main categories:

- Gaseous Pollutants
- Particulate Pollutants

In [ ]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#
#

Next, we need to find if there are any Air monitoring stations close to Lubbock. This requuires the [FIPS](https://www.census.gov/library/reference/code-lists/ansi.html) number for the state and county as a 5 digit string along with latitude and longitdue info. We define it in the constant below

In [ ]:
# TX|48|303|01383937|Lubbock County|H1|A

CITY_LOCATIONS = {
    'lubbock' :       {'city'   : 'Lubbock',
                       'county' : 'Lubbock',
                       'state'  : 'Texas',
                       'fips'   : '48303',
                       'latlon' : [34.12, -117.39] }
}


In [ ]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city selected from the CITY_LOCATIONS dictionary
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['lubbock']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['lubbock']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "0001",
        "value_represented": "NE OF 5TH STREET & AVENUE K INTERSECTION"
    },
    {
        "code": "0002",
        "value_represented": null
    },
    {
        "code": "0003",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "0005",
        "value_represented": null
    },
    {
        "code": "0006",
        "value_represented": null
    },
    {
        "code": "0007",
        "value_represented": null
    },
    {
        "code": "0008",
        "value_represented": null
    },
    {
        "code": "0009",
        "value_represented": null
    },
    {
        "code": "0010",
        "value_represented": null
    },
    {
        "code": "0011",
        "value_represented": null
    },
    {
        "code": "0012",
        "value_represented": null
    },
    {
        "code": "0013",
        "value_represented": null
    },
    {
        "code": "0014",
       

We do get some results for monitoring stations close to Lubbock so we are good to go.

## Making a daily summary request

We next define a function to get a daily summary request fromt the API and another function to parse the results.

In [ ]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date.
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL,
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY,
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):

    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']:
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']:
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']:
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [ ]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_GASEOUS
request_data['state'] = CITY_LOCATIONS['lubbock']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['lubbock']['fips'][2:]

# request daily summary data for the month of July in 2021
gaseous_aqi = request_daily_summary(request_template=request_data, begin_date="20210701", end_date="20210731")
print("Response for the gaseous pollutants ...")
#
if gaseous_aqi["Header"][0]['status'] == "Success":
    print(json.dumps(gaseous_aqi['Data'],indent=4))
elif gaseous_aqi["Header"][0]['status'].startswith("No data "):
    print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
else:
    print(json.dumps(gaseous_aqi,indent=4))

Response for the gaseous pollutants ...
Looks like the response generated no data. You might take a closer look at your request and the response data.


In [ ]:
request_data['param'] = AQI_PARAMS_PARTICULATES
# request daily summary data for the month of July in 2021
particulate_aqi = request_daily_summary(request_template=request_data, begin_date="20210701", end_date="20210731")
print("Response for the particulate pollutants ...")
#
if particulate_aqi["Header"][0]['status'] == "Success":
    print(json.dumps(particulate_aqi['Data'],indent=4))
elif particulate_aqi["Header"][0]['status'].startswith("No data "):
    print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
else:
    print(json.dumps(particulate_aqi,indent=4))

Response for the particulate pollutants ...
[
    {
        "state_code": "48",
        "county_code": "303",
        "site_number": "1028",
        "parameter_code": "88101",
        "poc": 1,
        "latitude": 33.58553,
        "longitude": -101.78698,
        "datum": "NAD83",
        "parameter": "PM2.5 - Local Conditions",
        "sample_duration_code": "1",
        "sample_duration": "1 HOUR",
        "pollutant_standard": null,
        "date_local": "2021-07-01",
        "units_of_measure": "Micrograms/cubic meter (LC)",
        "event_type": "No Events",
        "observation_count": 22,
        "observation_percent": 92.0,
        "validity_indicator": "Y",
        "arithmetic_mean": 5.5,
        "first_max_value": 9.0,
        "first_max_hour": 20,
        "aqi": null,
        "method_code": "209",
        "method": "Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM2.5C - Beta Attenuation",
        "local_site_name": "Lubbock 12th Street",
        "site_address": "3901 East 12

Since the output from the function is quite lengthy, we will define another function to extract quantites of interest from the results.

In [ ]:
#
#    This is a list of field names - data - that will be extracted from each record
#
EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']

#
#    The function creates a summary record
def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
    ## the result will be structured around monitoring site, parameter, and then date
    result = dict()
    data = r["Data"]
    for record in data:
        # make sure the record is set up
        site = record['site_number']
        param = record['parameter_code']
        #date = record['date_local']    # this version keeps the respnse value YYYY-
        date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
        if site not in result:
            result[site] = dict()
            result[site]['local_site_name'] = record['local_site_name']
            result[site]['site_address'] = record['site_address']
            result[site]['state'] = record['state']
            result[site]['county'] = record['county']
            result[site]['city'] = record['city']
            result[site]['pollutant_type'] = dict()
        if param not in result[site]['pollutant_type']:
            result[site]['pollutant_type'][param] = dict()
            result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
            result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
            result[site]['pollutant_type'][param]['method'] = record['method']
            result[site]['pollutant_type'][param]['data'] = dict()
        if date not in result[site]['pollutant_type'][param]['data']:
            result[site]['pollutant_type'][param]['data'][date] = list()

        # now extract the specified fields
        extract = dict()
        for k in fields:
            if str(k) in record:
                extract[str(k)] = record[k]
            else:
                # this makes sure we always have the requested fields, even if
                # we have a missing value for a given day/month
                extract[str(k)] = None

        # add this extraction to the list for the day
        result[site]['pollutant_type'][param]['data'][date].append(extract)

    return result


In [ ]:

extract_gaseous = extract_summary_from_response(gaseous_aqi)
print("Summary of gaseous extraction ...")
print(json.dumps(extract_gaseous,indent=4))

extract_particulate = extract_summary_from_response(particulate_aqi)
print("Summary of particulate extraction ...")
print(json.dumps(extract_particulate,indent=4))


Summary of gaseous extraction ...
{}
Summary of particulate extraction ...
{
    "1028": {
        "local_site_name": "Lubbock 12th Street",
        "site_address": "3901 East 12th Street",
        "state": "Texas",
        "county": "Lubbock",
        "city": "Lubbock",
        "pollutant_type": {
            "88101": {
                "parameter_name": "PM2.5 - Local Conditions",
                "units_of_measure": "Micrograms/cubic meter (LC)",
                "method": "Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM2.5C - Beta Attenuation",
                "data": {
                    "20210701": [
                        {
                            "sample_duration": "1 HOUR",
                            "observation_count": 22,
                            "arithmetic_mean": 5.5,
                            "aqi": null
                        },
                        {
                            "sample_duration": "24-HR BLK AVG",
                            "observation_cou

##  Making request by bounding box

Upon examing the summary, we find that there is no data for AQI gaseous pollutants. In other to see if we can get more data, we use a bounding box (as in a radius around our target city) to see if we can get some data for a wider area.

In [ ]:
#
#   These are rough estimates for creating bounding boxes based on a city location
#   You can find these rough estimates on the USGS website:
#   https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps
#
LAT_25MILES = 25.0 * (1.0/69.0)    # This is about 25 miles of latitude in decimal degrees
LON_25MILES = 25.0 * (1.0/54.6)    # This is about 25 miles of longitude in decimal degrees
#
#   Compute rough estimates for a bounding box around a given place
#   The bounding box is scaled in 50 mile increments. That is, the bounding box will have sides that
#   are rough multiples of 50 miles, with the center of the box around the indicated place.
#   The scale parameter determines the scale (size) of the bounding box
#
def bounding_latlon(place=None,scale=1.0):
    minlat = place['latlon'][0] - float(scale) * LAT_25MILES
    maxlat = place['latlon'][0] + float(scale) * LAT_25MILES
    minlon = place['latlon'][1] - float(scale) * LON_25MILES
    maxlon = place['latlon'][1] + float(scale) * LON_25MILES
    return [minlat,maxlat,minlon,maxlon]



In [ ]:
#
#    This implements the monitors request. This requests monitoring stations. This can be done by state, county, or bounding box.
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_monitors(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL,
                          endpoint_action = API_ACTION_MONITORS_COUNTY,
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):

    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_monitors()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_monitors()'")
    if not request_template['param']:
        raise Exception("Must supply param values to call 'request_monitors()'")
    if not request_template['begin_date']:
        raise Exception("Must supply a begin_date to call 'request_monitors()'")
    if not request_template['end_date']:
        raise Exception("Must supply an end_date to call 'request_monitors()'")
    # Note we're not validating FIPS fields because not all of the monitors actions require the FIPS numbers

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [ ]:
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_PARTICULATES     # remember we have both gaseous and particulates
#

request_data['state'] = CITY_LOCATIONS['lubbock']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['lubbock']['fips'][2:]
#
# the first example uses the default - request monitors by county, we'll just use a recent date for now
response = request_monitors(request_template=request_data, begin_date="20210701", end_date="20210731")
#
# the response should be similar to the 'list' request above - but in this case we should only get monitors that
# monitor the AQI_PARAMS_PARTICULATES set of params.
#
if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "state_code": "48",
        "county_code": "303",
        "site_number": "1028",
        "parameter_code": "88101",
        "poc": 1,
        "parameter_name": "PM2.5 - Local Conditions",
        "open_date": "2018-07-11",
        "close_date": null,
        "concurred_exclusions": null,
        "dominant_source": null,
        "measurement_scale": "URBAN SCALE",
        "measurement_scale_def": "4 KM TO 50 KM",
        "monitoring_objective": "POPULATION EXPOSURE",
        "last_method_code": "209",
        "last_method_description": "Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM2.5C - Beta Attenuation",
        "last_method_begin_date": "2018-07-11",
        "naaqs_primary_monitor": "Y",
        "qa_primary_monitor": null,
        "monitor_type": "SPM",
        "networks": null,
        "monitoring_agency_code": "1035",
        "monitoring_agency": "Texas Commission On Environmental Quality",
        "si_id": 102078,
        "latitude": 33.58553,
        "longitude":

## Getting data for our required timeframe

Now that we know the structure of the responses, we are ready to make the requests to get the data from 1961 to 2021 for the fire season. We will compare the extracted AQI values with the calcualted smoke estimates (present in the other notebook on this repo)
We then process our JSON data into a dataframe, aggregate them to find yearly AQI (since our smoke estimates are yearly) and then write it to a CSV.

We notice that we don't have a lot of AQI gaseous pollutant data for Lubbock which we try to estimate using the AQI values from the particulate component and the arthimetic mean of the gaseous component. If we don't have any information for that year at all, we fill the null values with zeros.

In [ ]:
START_MMDD = "0501"
END_MMDD = "1031"

# FIPS
STATE_FIPS = '48'
COUNTY_FIPS = '303'
YEAR_START = 1961
YEAR_END = 2021

In [ ]:
def extract_AQI_data(aqi_JSON):
    """
    Extracts AQI Data from JSON response if the request is successful.

    Parameters:
        aqi_JSON (dict): The raw JSON response of the request.
    Return:
        list: the list of dictionaries denoting each sensor's measurement for a day.
    """
    if aqi_JSON["Header"][0]['status'] == "Success":
        return aqi_JSON['Data']
    elif aqi_JSON["Header"][0]['status'].startswith("No data "):
#         print("Looks like the response generated no data. You might take a closer look at your request and the response data.")
        pass
    else:
        print("Ruh-Roh! Something's happening. Take a look!")
        print(json.dumps(aqi_JSON,indent=4))
    return None

In [ ]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = STATE_FIPS
request_data['county'] = COUNTY_FIPS
bbox = bounding_latlon(CITY_LOCATIONS['lubbock'],scale=3.0)
#   150 mile box - roughly within 75 miles of the location
#bbox = bounding_latlon(CITY_LOCATIONS['seaside'],scale=3.0)
#   200 mile box
#bbox = bounding_latlon(CITY_LOCATIONS['seaside'],scale=4.0)

# the bbox response comes back as a list - [minlat,maxlat,minlon,maxlon]

#   put our bounding box into the request_data
request_data['minlat'] = bbox[0]
request_data['maxlat'] = bbox[1]
request_data['minlon'] = bbox[2]
request_data['maxlon'] = bbox[3]

gaseous_responses = []
particulate_responses = []

# We'll iterate through all the years
for year in trange(YEAR_START, YEAR_END + 1):
    # Define start and end times for the fire season
    begin_date = str(year) + START_MMDD
    end_date = str(year) + END_MMDD

    # Request daily summary data for gaseous pollutants
    request_data['param'] = AQI_PARAMS_GASEOUS
    gaseous_aqi = request_daily_summary(
        request_template=request_data, begin_date=begin_date, end_date=end_date)
    gas_list = extract_AQI_data(gaseous_aqi)

    # Request daily summary data for particulate pollutants
    request_data['param'] = AQI_PARAMS_PARTICULATES
    particulate_aqi = request_daily_summary(
        request_template=request_data, begin_date=begin_date, end_date=end_date)
    part_list = extract_AQI_data(particulate_aqi)

    # add responses to larger set
    if gas_list:
        gaseous_responses.extend(gas_list)
    else:
        print(f"No gaseous data available for {year}.")

    if part_list:
        particulate_responses.extend(part_list)
    else:
        print(f"No particulate data available for {year}.")

  0%|          | 0/61 [00:00<?, ?it/s]

No gaseous data available for 1961.
No particulate data available for 1961.
No gaseous data available for 1962.
No particulate data available for 1962.
No gaseous data available for 1963.
No particulate data available for 1963.
No gaseous data available for 1964.
No particulate data available for 1964.
No gaseous data available for 1965.
No particulate data available for 1965.
No gaseous data available for 1966.
No particulate data available for 1966.
No gaseous data available for 1967.
No particulate data available for 1967.
No gaseous data available for 1968.
No particulate data available for 1968.
No particulate data available for 1969.
No particulate data available for 1970.
No particulate data available for 1971.
No particulate data available for 1972.
No particulate data available for 1973.
No particulate data available for 1974.
No particulate data available for 1975.
No particulate data available for 1976.
No particulate data available for 1977.
No particulate data available fo

In [ ]:
# If there is gaseous AQI data available, convert & store it as csv.
if gaseous_responses:
    gas_df = pd.DataFrame.from_records(gaseous_responses)
    gas_df.to_csv("gaseous_AQI_1963-2023.csv")
else:
    print("There were no gaseous AQI data available to store.")
# if there is particulate AQI data available, convert and store it as csv.
if particulate_responses:
    part_df = pd.DataFrame.from_records(particulate_responses)
    part_df.to_csv("particulate_AQI_1963-2023.csv")
else:
    print("There were no particulate AQI data available to store.")

In [ ]:
gas_df.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,48,303,0001,42401,1,33.590851,-101.847594,WGS84,Sulfur dioxide,7,...,091,GAS-BUBBLER - PARAROSANILINE-SULFAMIC ACID,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2013-06-11
1,48,303,0001,42401,1,33.590851,-101.847594,WGS84,Sulfur dioxide,7,...,091,GAS-BUBBLER - PARAROSANILINE-SULFAMIC ACID,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2013-06-11
2,48,303,0001,42401,1,33.590851,-101.847594,WGS84,Sulfur dioxide,7,...,091,GAS-BUBBLER - PARAROSANILINE-SULFAMIC ACID,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2013-06-11
3,48,303,0001,42401,1,33.590851,-101.847594,WGS84,Sulfur dioxide,7,...,091,GAS-BUBBLER - PARAROSANILINE-SULFAMIC ACID,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2013-06-11
4,48,303,0001,42401,1,33.590851,-101.847594,WGS84,Sulfur dioxide,7,...,091,GAS-BUBBLER - PARAROSANILINE-SULFAMIC ACID,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2013-06-11


In [ ]:
part_df.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
1,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
2,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
3,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
4,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22


In [ ]:
# let's only extract the non-null AQI
aqi_gas=gas_df[~gas_df['arithmetic_mean'].isna()]
aqi_part=part_df[~part_df['aqi'].isna()]
#.head().style

In [ ]:
aqi_gas.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration_code',
       'sample_duration', 'pollutant_standard', 'date_local',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code', 'method',
       'local_site_name', 'site_address', 'state', 'county', 'city',
       'cbsa_code', 'cbsa', 'date_of_last_change'],
      dtype='object')

In [ ]:
aqi_part.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
1,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
2,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
3,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22
4,48,303,0001,81102,1,33.590851,-101.847594,WGS84,PM10 Total 0-10um STP,7,...,052,HI-VOL-SA321A - GRAVIMETRIC,NE OF 5TH STREET & AVENUE K INTERSECTION,5TH STREET AT AVE. K,Texas,Lubbock,Lubbock,31180,"Lubbock, TX",2024-05-22


In [ ]:
import pandas as pd

# Assume df_gas is the DataFrame for gaseous components, missing AQI values
# Assume df_pm25 is the DataFrame containing PM2.5 data with valid AQI values as a reference

# Filter df_pm25 to contain only rows with PM2.5 data and valid AQI
aqi_part = aqi_part[aqi_part['parameter'] == 'PM2.5 - Local Conditions'].dropna(subset=['aqi'])

# Extract unique concentration and AQI values from df_pm25
aqi_part_reference = aqi_part[['arithmetic_mean', 'aqi']].drop_duplicates()

# Sort by concentration for interpolation
aqi_part_reference = aqi_part_reference.sort_values(by='arithmetic_mean').reset_index(drop=True)

# Function to interpolate AQI based on concentration using PM2.5 data
def interpolate_aqi(concentration, reference_df):
    if concentration < reference_df['arithmetic_mean'].min() or concentration > reference_df['arithmetic_mean'].max():
        return None  # Return None if out of range
    # Interpolate AQI for given concentration
    return round(np.interp(concentration, reference_df['arithmetic_mean'], reference_df['aqi']))

# Apply interpolation to df_gas for rows with missing AQI
aqi_gas['aqi'] = aqi_gas.apply(
    lambda row: interpolate_aqi(row['arithmetic_mean'], aqi_part_reference) if pd.isnull(row['aqi']) else row['aqi'], axis=1
)

# Display the updated DataFrame with calculated AQI values
print(aqi_gas[['parameter', 'arithmetic_mean', 'aqi']].head())


        parameter  arithmetic_mean   aqi
0  Sulfur dioxide              1.5   8.0
1  Sulfur dioxide              1.5   8.0
2  Sulfur dioxide              3.4  19.0
3  Sulfur dioxide              3.4  19.0
4  Sulfur dioxide              1.5   8.0


In [ ]:
useful_columns = [
    'site_number',
    'parameter_code',
    'latitude',
    'longitude',
    'datum',
    'parameter',
    'sample_duration',
    'date_local',
    'units_of_measure',
    'arithmetic_mean',
    'aqi',
]
aqi_gas = aqi_gas[useful_columns]
aqi_part = aqi_part[useful_columns]

In [ ]:
aqi_gas

,site_number,parameter_code,latitude,longitude,datum,parameter,sample_duration,date_local,units_of_measure,arithmetic_mean,aqi
0,0001,42401,33.590851,-101.847594,WGS84,Sulfur dioxide,24 HOUR,1969-05-20,Parts per billion,1.5,8.0
1,0001,42401,33.590851,-101.847594,WGS84,Sulfur dioxide,24 HOUR,1969-05-20,Parts per billion,1.5,8.0
2,0001,42401,33.590851,-101.847594,WGS84,Sulfur dioxide,24 HOUR,1969-07-18,Parts per billion,3.4,19.0
3,0001,42401,33.590851,-101.847594,WGS84,Sulfur dioxide,24 HOUR,1969-07-18,Parts per billion,3.4,19.0
4,0001,42401,33.590851,-101.847594,WGS84,Sulfur dioxide,24 HOUR,1969-07-29,Parts per billion,1.5,8.0
...,...,...,...,...,...,...,...,...,...,...,...
657,0001,42602,33.590851,-101.847594,WGS84,Nitrogen dioxide (NO2),24 HOUR,1980-09-30,Parts per billion,9.0,50.0
658,0001,42602,33.590851,-101.847594,WGS84,Nitrogen dioxide (NO2),24 HOUR,1980-10-06,Parts per billion,11.6,56.0
659,0001,42602,33.590851,-101.847594,WGS84,Nitrogen dioxide (NO2),24 HOUR,1980-10-12,Parts per billion,13.2,59.0
660,0001,42602,33.590851,-101.847594,WGS84,Nitrogen dioxide (NO2),24 HOUR,1980-10-18,Parts per billion,8.4,47.0


In [ ]:
aqi_part

,site_number,parameter_code,latitude,longitude,datum,parameter,sample_duration,date_local,units_of_measure,arithmetic_mean,aqi
1261,0001,88101,33.590851,-101.847594,WGS84,PM2.5 - Local Conditions,24 HOUR,1999-07-05,Micrograms/cubic meter (LC),5.1,28.0
1262,0001,88101,33.590851,-101.847594,WGS84,PM2.5 - Local Conditions,24 HOUR,1999-07-05,Micrograms/cubic meter (LC),5.1,28.0
1263,0001,88101,33.590851,-101.847594,WGS84,PM2.5 - Local Conditions,24 HOUR,1999-07-05,Micrograms/cubic meter (LC),5.1,28.0
1264,0001,88101,33.590851,-101.847594,WGS84,PM2.5 - Local Conditions,24 HOUR,1999-07-05,Micrograms/cubic meter (LC),5.1,28.0
1265,0001,88101,33.590851,-101.847594,WGS84,PM2.5 - Local Conditions,24 HOUR,1999-07-05,Micrograms/cubic meter (LC),5.1,28.0
...,...,...,...,...,...,...,...,...,...,...,...
12382,1028,88101,33.585530,-101.786980,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,2021-10-31,Micrograms/cubic meter (LC),3.7,21.0
12383,1028,88101,33.585530,-101.786980,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,2021-10-31,Micrograms/cubic meter (LC),3.7,21.0
12384,1028,88101,33.585530,-101.786980,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,2021-10-31,Micrograms/cubic meter (LC),3.7,21.0
12385,1028,88101,33.585530,-101.786980,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,2021-10-31,Micrograms/cubic meter (LC),3.7,21.0


In [ ]:
aqi_gas.value_counts(subset = ['parameter', 'sample_duration']).head()

,,count
parameter,sample_duration,
Sulfur dioxide,24 HOUR,470
Nitrogen dioxide (NO2),24 HOUR,192


In [ ]:
aqi_part.value_counts(subset = ['parameter', 'sample_duration']).head()

parameter                 sample_duration
PM2.5 - Local Conditions  24-HR BLK AVG      5048
                          24 HOUR            2504
Name: count, dtype: int64

In [ ]:
final_aqi_data = pd.concat([aqi_gas, aqi_part])
final_aqi_data.to_csv("AQI_data_1961-2021.csv")

In [ ]:
final_daily_estimate = final_aqi_data[['date_local', 'aqi']].groupby(by='date_local').max().reset_index()
final_daily_estimate['date_local'] = pd.to_datetime(final_daily_estimate['date_local'])
final_daily_estimate = final_daily_estimate.sort_values(by='date_local').reset_index(drop=True)

In [ ]:
final_daily_estimate['year'] = final_daily_estimate.date_local.dt.year
final_annual_estimate = final_daily_estimate[['year', 'aqi']].groupby('year').mean().reset_index()

In [ ]:
final_annual_estimate.to_csv('yearly_AQI_1961-2021.csv')